In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/acl-codemixing-dataset/sentNob/sentNob_dataset.csv
/kaggle/input/acl-codemixing-dataset/xl_sum/xl_sum_dataset.csv
/kaggle/input/acl-codemixing-dataset/nmt/nmt_en.csv
/kaggle/input/acl-codemixing-dataset/nmt/nmt_dataset.csv
/kaggle/input/acl-codemixing-dataset/fake_news/fake_news_dataset.csv
/kaggle/input/acl-codemixing-dataset/hate/hate_dataset.csv


In [2]:
!pip install anthropic -q

In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
claude_key = user_secrets.get_secret("claude_key_fabiha")

In [4]:
import os
from tqdm import tqdm
import pandas as pd
import json
from sklearn.metrics import accuracy_score
import anthropic
import time

client = anthropic.Anthropic(
    api_key=claude_key,
)

In [5]:
total_cost = 0

In [6]:
def get_sentiment_analysis(data, system, prefill=""):
    max_retries = 3
    retry_delay = 60  # 60 seconds wait between retries
    
    for attempt in range(max_retries):
        try:
            message = client.messages.create(
                model="claude-3-5-sonnet-20241022",
                max_tokens=2000,
                temperature=0.0,
                system=system,
                messages=[
                    {"role": "user", "content": data},
                    {"role": "assistant", "content": prefill}
                ]
            )
            
            input_tokens = message.usage.input_tokens
            output_tokens = message.usage.output_tokens
            
            global total_cost
            input_cost = input_tokens * 0.000003
            output_cost = output_tokens * 0.000015
            total_cost += input_cost + output_cost
            # print(message.content[0].text)
            response_text = message.content[0].text
            with open('sentences.txt', 'a', encoding='utf-8') as f:
                f.write(response_text + '\n')
            print(response_text)
            return response_text
            
        except anthropic.RateLimitError:
            if attempt < max_retries - 1:
                print(f"Rate limit hit. Waiting {retry_delay} seconds before retry...")
                time.sleep(retry_delay)
            else:
                raise

### bangla prompt

In [7]:
base_prompt = '''
You are an expert in natural language processing and text summarization. Your task is to summarize Bengali text into a concise and meaningful version while preserving the key points and overall meaning.

Follow these steps:
1. Read the input Bengali text carefully.
2. Identify the main ideas, key points, and essential information.
3. Write a summary that is shorter than the original text but retains the core meaning.

Ensure:
- Complete JSON structure
- No truncation
- Valid syntax

You will receive an array of samples, each containing "id" and "text".
'''

output_prompt = ' Use JSON format with the keys as \"id\", \"pred_summary\". Return only the JSON object with the root named "samples". Do not change the value of the "id".'

bangla_prompt = base_prompt + output_prompt


### codemixed prompt

In [8]:
base_prompt = '''
You are an expert in natural language processing and text summarization. Your task is to summarize codemixed Bengali text into a concise and meaningful version while preserving the key points and overall meaning.

Follow these steps:
1. Read the input Bengali text carefully.
2. Identify the main ideas, key points, and essential information.
3. Write a summary that is shorter than the original text but retains the core meaning.

Ensure:
- Complete JSON structure
- No truncation
- Valid syntax

You will receive an array of samples, each containing "id" and "text".
'''

output_prompt = ' Use JSON format with the keys as \"id\", \"pred_summary\". Return only the JSON object with the root named "samples". Do not change the value of the "id".'

codemixed_prompt = base_prompt + output_prompt


In [9]:
df = pd.read_csv('/kaggle/input/acl-codemixing-dataset/xl_sum/xl_sum_dataset.csv')

df = df.sample(n=200, random_state=42) 
# df = df[:200]

In [10]:
df

,id,url,title,summary,text,text_length_char,text_length_word,sentence_count,rand_word_idx,rand_sent_idx,salient_words,salient_words_scores,perturbed_text_words,perturbed_text_sentences,perturbed_text_salient
631,news-55965015,https://www.bbc.com/bengali/news-55965015,করোনা ভাইরাস ভ্যাকসিন: দেশব্যাপি টিকাদান শুরু ...,বাংলাদেশে আজ (রবিবার) থেকে শুরু হতে যাচ্ছে করো...,৭ই ফেব্রুয়ারি থেকে বাংলাদেশের সব জেলা উপজেলায...,5049,766,53,"[448, 605, 98, 86, 750, 731, 537, 558, 231, 69...","[40, 15, 2, 13, 34, 26, 18, 37, 7, 17, 25]","['অ্যাস্ট্রেজেনেকার', 'বেশ', 'পার্শ্বপ্রতিক্রি...","[0.009495888, 0.009019518, 0.0082483, 0.007195...",৭ই ফেব্রুয়ারি থেকে বাংলাদেশের সব জেলা উপজেলায...,৭ই ফেব্রুয়ারি থেকে বাংলাদেশের সব জেলা উপজেলায...,৭ই ফেব্রুয়ারি থেকে বাংলাদেশের sob জেলা Upazil...
634,news-51532541,https://www.bbc.com/bengali/news-51532541,বোরকা বিতর্ক: তসলিমা নাসরিনের আক্রমণের জবাবে য...,ভারতের খ্যাতিমান সঙ্গীত পরিচালক এ আর রহমানের ক...,এ আর রহমানের কন্যা খাতিজা রহমান (বামে) এবং তসল...,3963,659,41,"[469, 378, 45, 326, 293, 473, 533, 25, 228, 52...","[6, 34, 33, 19, 17, 16, 5, 0]","['স্ক্রীনশটসহ', 'আম্বানির', 'আম্বানি', 'কোলহফ'...","[0.007951893, 0.0065853093, 0.0061852243, 0.00...",এ আর রহমানের কন্যা খাতিজা Rahman (বামে) এবং Ta...,A R Rahmaner Kanya Khatija Rahman (Bame) ebong...,a r রহমানের daughter Khatija Rahman ( বামে ) a...
963,news-44466636,https://www.bbc.com/bengali/news-44466636,ট্রাম্প-কিম বৈঠক: দিনের কিছু আলোচিত মুহূর্ত,মার্কিন প্রেসিডেন্ট ডোনাল্ড ট্রাম্পের সাথে উত্...,উত্তর কোরিয়ার একটি সমুদ্র সৈকতের ছবি। তাদের ম...,3284,526,42,"[290, 274, 101, 481, 520, 83, 172, 67, 120, 20...","[28, 23, 30, 9, 24, 1, 3, 21]","['ফটোসাংবাদিকদের', 'লিমুজিন', 'সকৌতুকে', 'পোজ'...","[0.009623764, 0.006113166, 0.00609175, 0.00500...",উত্তর কোরিয়ার akti সমুদ্র সৈকতের ছবি। tader ম...,উত্তর কোরিয়ার একটি সমুদ্র সৈকতের ছবি।Tader mo...,north কোরিয়ার akti সমুদ্র সৈকতের picture । তা...
625,news-55821540,https://www.bbc.com/bengali/news-55821540,বাংলাদেশে অপরাধ না করেও বছরের পর বছর কারাগারে ...,বাংলাদেশের সম্প্রতি দুর্নীতি দমন কমিশনের করা এ...,বাংলাদেশে বিনা দোষে বছরের পর বছর কারাদণ্ড ভোগ ...,5216,839,48,"[101, 175, 436, 192, 158, 509, 442, 93, 497, 6...","[35, 38, 45, 19, 9, 10, 23, 18, 40, 8]","['বেশ', 'বেরানব্বই', 'জাহালম', 'দীর্ঘসূত্রিতার...","[0.008787271, 0.008297306, 0.0062711816, 0.005...",in Bangladesh বিনা দোষে year পর বছর কারাদণ্ড ভ...,বাংলাদেশে বিনা দোষে বছরের পর বছর কারাদণ্ড ভোগ ...,বাংলাদেশে bina দোষে year por year কারাদণ্ড ভোগ...
365,news-43735107,https://www.bbc.com/bengali/news-43735107,গেজেট না হওয়া পর্যন্ত কোটা সংস্কার আন্দোলন স্...,বাংলাদেশের প্রধানমন্ত্রী শেখ হাসিনা সরকারি চাক...,ঢাকা বিশ্ববিদ্যালয়ের সমবেত আন্দোলনকারীরা ঢাকা...,2809,388,19,"[207, 324, 214, 85, 184, 267, 196, 219, 356, 1...","[13, 2, 1, 0]","['অনগ্রসরদের', 'তাৎক্ষনিক', 'বাড়লো', 'সংকটকে'...","[0.009979705, 0.006460202, 0.0053381203, 0.005...",ঢাকা বিশ্ববিদ্যালয়ের সমবেত আন্দোলনকারীরা ঢাকা...,Dhaka university er somporkito advocates dhaka...,Dhaka বিশ্ববিদ্যালয়ের সমবেত agitators Dhaka ব...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,news-38298662,https://www.bbc.com/bengali/news-38298662,বিদেশী শিক্ষার্থী ভর্তি কমাচ্ছে ব্রিটেন,যুক্তরাজ্যের বিশ্ববিদ্যালয়গুলোতে পড়তে যাওয়া...,যুক্তরাজ্যে প্রতিবছর তিন লক্ষ বিদেশী শিক্ষার্থ...,1548,209,12,"[157, 24, 123, 74, 179, 16, 163, 155, 97, 203,...","[10, 3]","['শেফিল্ড', 'কনজারভেটিভ', 'রিওর্ডানকে', 'ব্রিট...","[0.0119792335, 0.010613035, 0.010524901, 0.009...",যুক্তরাজ্যে প্রতিবছর তিন লক্ষ বিদেশী শিক্ষার্থ...,যুক্তরাজ্যে প্রতিবছর তিন লক্ষ বিদেশী শিক্ষার্থ...,যুক্তরাজ্যে প্রতিবছর তিন lakh বিদেশী শিক্ষার্থ...
208,news-38824092,https://www.bbc.com/bengali/news-38824092,স্কুল শিক্ষার্থীদের শরীর সেতুর ওপর চেয়ারম্যান,দুই সারিতে স্কুল শিক্ষার্থীরা দাঁড়িয়ে মেলে দ...,এই ছবিটি ঘুরে বেড়াচ্ছে সামাজিক যোগাযোগ মাধ্যম...,2391,364,32,"[167, 263, 363, 184, 10, 346, 27, 59, 353, 229...","[28, 22, 13, 17, 29, 3]","['উছমানিয়া', 'শিক্ষর্থীর', 'শিক্ষর্থী', 'জোরা...","[0.008297925, 0.007971245, 0.007505936, 0.0065..."

In [11]:
df = df.reset_index(drop=True)
df['id'] = df.index + 1

In [12]:
df.columns

Index(['id', 'url', 'title', 'summary', 'text', 'text_length_char',
       'text_length_word', 'sentence_count', 'rand_word_idx', 'rand_sent_idx',
       'salient_words', 'salient_words_scores', 'perturbed_text_words',
       'perturbed_text_sentences', 'perturbed_text_salient'],
      dtype='object')

In [13]:
# df['headline_content'] = df['headline'] + " " + df['content'] 

In [14]:
df.shape

(200, 15)

In [15]:
df.head(2)

,id,url,title,summary,text,text_length_char,text_length_word,sentence_count,rand_word_idx,rand_sent_idx,salient_words,salient_words_scores,perturbed_text_words,perturbed_text_sentences,perturbed_text_salient
0,1,https://www.bbc.com/bengali/news-55965015,করোনা ভাইরাস ভ্যাকসিন: দেশব্যাপি টিকাদান শুরু ...,বাংলাদেশে আজ (রবিবার) থেকে শুরু হতে যাচ্ছে করো...,৭ই ফেব্রুয়ারি থেকে বাংলাদেশের সব জেলা উপজেলায...,5049,766,53,"[448, 605, 98, 86, 750, 731, 537, 558, 231, 69...","[40, 15, 2, 13, 34, 26, 18, 37, 7, 17, 25]","['অ্যাস্ট্রেজেনেকার', 'বেশ', 'পার্শ্বপ্রতিক্রি...","[0.009495888, 0.009019518, 0.0082483, 0.007195...",৭ই ফেব্রুয়ারি থেকে বাংলাদেশের সব জেলা উপজেলায...,৭ই ফেব্রুয়ারি থেকে বাংলাদেশের সব জেলা উপজেলায...,৭ই ফেব্রুয়ারি থেকে বাংলাদেশের sob জেলা Upazil...
1,2,https://www.bbc.com/bengali/news-51532541,বোরকা বিতর্ক: তসলিমা নাসরিনের আক্রমণের জবাবে য...,ভারতের খ্যাতিমান সঙ্গীত পরিচালক এ আর রহমানের ক...,এ আর রহমানের কন্যা খাতিজা রহমান (বামে) এবং তসল...,3963,659,41,"[469, 378, 45, 326, 293, 473, 533, 25, 228, 52...","[6, 34, 33, 19, 17, 16, 5, 0]","['স্ক্রীনশটসহ', 'আম্বানির', 'আম্বানি', 'কোলহফ'...","[0.007951893, 0.0065853093, 0.0061852243, 0.00...",এ আর রহমানের কন্যা খাতিজা Rahman (বামে) এবং Ta...,A R Rahmaner Kanya Khatija Rahman (Bame) ebong...,a r রহমানের daughter Khatija Rahman ( বামে ) a...


## choose val or test dataset

In [16]:
chunk_size = 5
chunks = [df[i:i + chunk_size] for i in range(0, len(df), chunk_size)]

output_dir = 'output/claude'
os.makedirs(output_dir, exist_ok=True)

## choose column

In [17]:
column = 'perturbed_text_sentences'

In [18]:
# def process_text(text):
#     return text.replace('\n', ' ')

In [19]:
def process_text(text):
    # Replace newlines with spaces and normalize multiple spaces
    processed = text.replace('\n', ' ').strip()
    while '  ' in processed:
        processed = processed.replace('  ', ' ')
    return processed

def create_chunks(df, column, chunk_size=10):
    chunk_list = []
    
    # Calculate number of chunks
    total_rows = len(df)
    num_chunks = (total_rows + chunk_size - 1) // chunk_size
    
    for chunk_idx in range(num_chunks):
        start_idx = chunk_idx * chunk_size
        end_idx = min((chunk_idx + 1) * chunk_size, total_rows)
        
        # Get chunk of DataFrame
        chunk_df = df.iloc[start_idx:end_idx]
        
        # Create formatted text entries
        text_entries = []
        for _, row in chunk_df.iterrows():
            clean_text = process_text(str(row[column]))
            text_entries.append(f"{row.name}: {clean_text}")
        
        # Create chunk dictionary
        chunk_dict = {
            'chunk_name': f"{chunk_df.index[0]}-{chunk_df.index[-1]}",
            'user_prompt': '\n'.join(text_entries)
        }
        
        chunk_list.append(chunk_dict)
    
    return chunk_list

In [20]:
chunk_list = create_chunks(df, column, chunk_size=5)

In [ ]:
# chunk_list[0]

In [ ]:
# chunk_list[221]

In [21]:
final_df = pd.DataFrame()
for chunk in chunk_list[:1]:
    try:
        data = chunk['user_prompt']
        response = get_sentiment_analysis(codemixed_prompt, data)
        parsed_data = json.loads(response)
        results = parsed_data['samples']
        
        output_df = pd.DataFrame(results)
        final_df = pd.concat([final_df, output_df], ignore_index=True)
        
        # Add a small delay between chunks to avoid rate limits
        time.sleep(2)  # 2 second delay between chunks
        
    except Exception as e:
        print(f"An error occurred for chunk {chunk['chunk_name']}: {e}")
        # Optionally save failed chunks for later retry
        continue

# final_df[id] = final_df.index + 1

An error occurred for chunk 0-4: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'You have reached your specified API usage limits. You will regain access on 2025-03-01 at 00:00 UTC.'}}


In [ ]:
print("total cost incurred: ", total_cost)

In [ ]:
# final_df

In [ ]:
final_df.shape

In [ ]:
# final_df.rename(columns = {'text': column}, inplace = True)
final_df.columns

In [ ]:
duplicates = final_df[final_df.duplicated(subset='id', keep=False)]
len(duplicates)

In [ ]:
final_df.drop_duplicates(subset='id', keep='first', inplace = True)
final_df.shape

In [ ]:
final_df

In [ ]:
df

In [ ]:
save_df = pd.merge(df, final_df, on = 'id', how = 'inner')
save_df

In [ ]:
# final_output_file = os.path.join(output_dir, f"claude_sentnob_{column}.csv")
final_output_file = f"claude_xl_sum_sentences.csv"
# final_df.to_csv(f"final_df_{final_output_file}", index=False)
save_df.to_csv(final_output_file, index=False)

print(f"Final DataFrame saved to {final_output_file}")